In [ ]:
import sagemaker
from utils import get_secret
from toy_datasets import upload_dataset_to_s3

Requierements unit 1 data

In [ ]:
session = sagemaker.Session()
role = get_secret('role_arn')
s3_bucket_uri = get_secret('s3_bucket_uri')
s3_bucket_name = get_secret('s3_bucket_name')

In [ ]:
image_uri = sagemaker.image_uris.retrieve('xgboost', region='us-east-1', version='1.5-1')

In [ ]:
from sagemaker.workflow.pipeline_context import PipelineSession

pipeline_session = PipelineSession()

In [ ]:
estimator = sagemaker.estimator.Estimator(
    image_uri=image_uri,
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    output_path=f"{s3_bucket_uri}/pipelines-output",
    sagemaker_session=pipeline_session
)

In [ ]:
estimator.set_hyperparameters(
    max_depth=5,
    objective='multi:softmax',
    num_class=3,
    num_round=10
)

In [ ]:
from sagemaker.inputs import TrainingInput

s3_train = TrainingInput(
    s3_data=f's3://{s3_bucket_name}/iris_dataset/train_data.csv',
    content_type='csv'
)

s3_validate = TrainingInput(
    s3_data=f's3://{s3_bucket_name}/iris_dataset/test_data.csv',
    content_type='csv'
)

In [ ]:
from sagemaker.workflow.steps import TrainingStep

training_step = TrainingStep(
    name='training-step',
    step_args=estimator.fit({
                'train': s3_train,
                'validation': s3_validate
    })
)

In [ ]:
from sagemaker.model import Model

model = Model(
    image_uri=image_uri,
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=pipeline_session,
    role=role
)

https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.register

In [ ]:
from sagemaker.workflow.model_step import ModelStep

register_params = {
    'model_package_group_name': 'iris-classification-group',
    'description': 'XGBoost model for Iris classification',
    'image_uri': image_uri,
    'task': 'CLASSIFICATION'
}


register_model_step = ModelStep(
    name='register-model-step',
    step_args=model.register(**register_params)
)

In [ ]:
from sagemaker.workflow.pipeline import Pipeline

pipeline = Pipeline(
    name='iris-classification-model-pipeline',
    steps=[training_step, register_model_step]
)

In [ ]:
parallelism_config={
    'MaxParallelExecutionSteps': 5
}

pipeline.create(
    role_arn=role,
    parallelism_config=parallelism_config,
    tags=[
        {'Key': 'Project', 'Value': 'AWSTutorials'},
        {'Key': 'Environment', 'Value': 'Development'}
    ]
)

In [ ]:
pipeline.start(
    execution_display_name='pipeline-execution',
    execution_description='First Pipeline Execution'
)

In [ ]:
import boto3
sm_client = boto3.client('sagemaker')

In [ ]:
# Call to list model packages
response = sm_client.list_model_packages(ModelPackageGroupName='iris-classification-group')

# Print the list of model packages
for model_package in response['ModelPackageSummaryList']:
    print(f"Model Package Group Name: {model_package['ModelPackageGroupName']}")
    print(f"Model Package ARN: {model_package['ModelPackageArn']}")
    print(f"Model Package Version: {model_package.get('ModelPackageVersion', 'N/A')}")
    print(f"Model Package Status: {model_package['ModelPackageStatus']}")
    print("------")

model_package_arn = response['ModelPackageSummaryList'][0]['ModelPackageArn']

In [ ]:
from sagemaker import ModelPackage
from sagemaker.serializers import CSVSerializer

model = ModelPackage(role=role, 
                     model_package_arn=model_package_arn, 
                     sagemaker_session=session)

predictor = model.deploy(initial_instance_count=1,
                     instance_type='ml.t2.medium',
                     endpoint_name='iris-endpointss',
                     serializer=CSVSerializer())

In [ ]:
predictor_two.predict([7.2, 3, 6, 1.6])

In [ ]:
predictor.predict([7.2, 3, 6, 1.6]).decode('utf-8').strip()

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor_two = sagemaker.Predictor(
    endpoint_name="iris-endpoints",
    serializer=CSVSerializer(),  # Adjust this based on the input your model expects
    deserializer=JSONDeserializer(),  # Adjust this based on the output format of your model
    sagemaker_session=session
)
